In [22]:
from sample import BaseModel, train_predictor, train_selector
from sample import PredictorNetwork
from sample import SelectorNetwork
import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [12]:
# Instantiate models, criterion, and optimizer
# base_model = BaseModel()
base_model = BaseModel()
base_model.eval()

# Load the pretrained ResNet-50 model
resnet50 = models.resnet50(pretrained=True)
# Switch to evaluation mode
resnet50.eval()

/Users/erik-luna/miniforge3/envs/cs221_hw7/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/erik-luna/miniforge3/envs/cs221_hw7/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
"""Set up the DataLoaders: """
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transformation for the validation data
transform = transforms.Compose([
    # transforms.Resize(256),
    # transforms.CenterCrop(224),
    # transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the validation dataset
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Files already downloaded and verified


In [23]:
"""Collect hidden layer output predictions"""
import torch
import torch.optim as optim
import torch.nn as nn
from collections import defaultdict

# Initialize lists to hold hidden layer outputs and final predictions
hidden_outputs = []
final_predictions = []

# Collect hidden layer outputs and final predictions
with torch.no_grad():
    for inputs, _ in val_loader:
        features = base_model(inputs)
        hidden_outputs.append(features)
        outputs = resnet50(inputs)
        final_predictions.append(outputs)

# Concatenate the collected data
hidden_outputs = torch.cat(hidden_outputs, dim=0)
final_predictions = torch.cat(final_predictions, dim=0)



In [24]:
from dataset_utils import PredictorDataset, SelectorDataset
# Example batch size
batch_size = 32

# Create the predictor dataset and dataloader
predictor_dataset = PredictorDataset(hidden_outputs, final_predictions)
predictor_data_loader = DataLoader(predictor_dataset, batch_size=batch_size, shuffle=True)

# Example predictor outputs and labels for the selector network
# These should be generated based on the predictor network's output and ground truth
predictor_outputs = torch.randn(len(hidden_outputs), 10)  # Example data, replace with actual predictor outputs
labels = torch.randint(0, 2, (len(hidden_outputs),))  # Example labels, replace with actual labels

# Create the selector dataset and dataloader
selector_dataset = SelectorDataset(predictor_outputs, labels)
selector_data_loader = DataLoader(selector_dataset, batch_size=batch_size, shuffle=True)

In [25]:

"""Train the Predictor and Selector Networks"""
predictor = PredictorNetwork(input_dim=512, output_dim=10)
selector = SelectorNetwork(input_dim=10)

criterion = nn.CrossEntropyLoss()
optimizer_pred = optim.Adam(predictor.parameters(), lr=0.001)
optimizer_sel = optim.Adam(selector.parameters(), lr=0.001)

num_epochs = 2
# Training loop
for epoch in range(num_epochs):
    train_predictor(predictor, predictor_data_loader, criterion, optimizer_pred)
    train_selector(selector, selector_data_loader, criterion, optimizer_sel)




RuntimeError: mat1 and mat2 shapes cannot be multiplied (21504x224 and 512x10)

****Testing the latency of the model****

In [ ]:
import time

def inference_with_cache(model, predictor, selector, data_loader, device):
    model.eval()
    predictor.eval()
    selector.eval()

    total_time = 0
    total_cache_hits = 0
    total_cache_misses = 0
    with torch.no_grad():
        for inputs in data_loader:
            inputs = inputs.to(device)
            
            start_time = time.time()

            # Get features from the base model
            features = model(inputs)
            
            # Use the predictor network to get predicted outputs
            predicted_outputs = predictor(features)

            # Use the selector network to determine cache hit/miss
            cache_decisions = selector(predicted_outputs).round()
            
            # Determine if cache hits or misses
            if cache_decisions.sum().item() > 0:
                cache_hits = cache_decisions.nonzero(as_tuple=True)[0]
                cache_misses = (1 - cache_decisions).nonzero(as_tuple=True)[0]

                # Cache hits: Use predicted outputs directly
                if len(cache_hits) > 0:
                    cached_outputs = predicted_outputs[cache_hits]
                    total_cache_hits += len(cached_outputs)
                
                # Cache misses: Run full model inference
                if len(cache_misses) > 0:
                    missed_inputs = inputs[cache_misses]
                    full_outputs = resnet50(missed_inputs)
                    total_cache_misses += len(full_outputs)
            
            else:
                # All misses: Run full model inference
                full_outputs = resnet50(inputs)
                total_cache_misses += len(full_outputs)
            
            end_time = time.time()
            total_time += (end_time - start_time)

    avg_latency = total_time / len(data_loader.dataset)
    return avg_latency, total_cache_hits, total_cache_misses
#setup the DataLoader for testing: 
# Define the transformation for the test data
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the test dataset
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transfer models to device
base_model.to(device)
predictor.to(device)
selector.to(device)

# Measure latency
avg_latency, cache_hits, cache_misses = inference_with_cache(base_model, predictor, selector, test_loader, device)
print(f"Average Latency with Learned Cache: {avg_latency:.6f} seconds")
print(f"Total Cache Hits: {cache_hits}")
print(f"Total Cache Misses: {cache_misses}")



In [ ]:
def inference_without_cache(model, data_loader, device):
    model.eval()
    
    total_time = 0
    with torch.no_grad():
        for inputs in data_loader:
            inputs = inputs.to(device)
            
            start_time = time.time()
            
            # Run full model inference
            outputs = resnet50(inputs)
            
            end_time = time.time()
            total_time += (end_time - start_time)

    avg_latency = total_time / len(data_loader.dataset)
    return avg_latency

# Measure latency without learned cache
baseline_latency = inference_without_cache(resnet50, test_loader, device)
print(f"Average Latency without Learned Cache: {baseline_latency:.6f} seconds")
